In [ ]:
# import modules
import re
import requests
import pandas as pd
from sec_web_scraper.Downloader import Downloader
from sec_web_scraper import get_document_given_link
from sec_edgar_downloader import Downloader
from bs4 import BeautifulSoup as bs
import sys
import unicodedata
from operator import itemgetter
import datetime
from datetime import date
from dateutil.relativedelta import relativedelta
from stocksent import Sentiment
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from pandas import json_normalize
from textblob import TextBlob

In [ ]:
# create request header
headers = {'User-Agent': "peter.j.1410@gmail.com"}

# get all companies data
companyTickers = requests.get(
    "https://www.sec.gov/files/company_tickers.json",
    headers=headers
    )

In [ ]:
#get the tickers for all public companies listed on USA exchanges
# parse CIK // without leading zeros
directCik = companyTickers.json()['0']['cik_str']

# dictionary to dataframe
companyData = pd.DataFrame.from_dict(companyTickers.json(),
                                     orient='index')

In [ ]:
# add leading zeros to CIK in order to perform future mappings
companyData['cik_str'] = companyData['cik_str'].astype(
                           str).str.zfill(10)

In [ ]:
companyData.head()

In [ ]:
# now we need to look for 8K filings for this companies and search for the bankruptcy section filings in this form
# Create new downloader object
#d = Downloader()

# input the year range for filing data
#d.build_index_sec(2016, 2022)

# get the dataframe with all 8-K's filled in the range above
#list_of_8k = d.find_files_by_type('8-K')

In [ ]:
list_of_8k

In [ ]:
#we use concatenation to obtain the full link for the each row of the table
sec_link = "https://www.sec.gov/Archives/"
list_of_8k['Filename'] = sec_link + list_of_8k['Filename'].astype(str)

In [ ]:
list_of_8k = list_of_8k.drop(columns=['url'])

In [ ]:
# we create a fuction to get the text from 8-K form with necessary items using regex
def parse_8k_filing(link):
    # retrieve the text file from SEC
    def get_text(link):
        page = requests.get(link, headers={'User-Agent': 'Mozilla'})
        html = bs(page.content, "lxml")
        text = html.get_text().replace(u'\xa0', ' ').replace("\t", " ").replace("\x92", "'").split("\n")
        return(text)
    # find items reported in 8-k
    def get_items(text):
        itemPattern = re.compile("^(Item\s[1-9][\.\d]*)")
        value = list()
        for line in text:
            if itemPattern.search(line.strip()) is not None:
                value.append(itemPattern.search(line.strip()).group(0))

        return(value)
    # get the text associated with the items
    def get_data(file, items) :
        dataList = list()
        stop = re.compile("SIGNATURE", re.IGNORECASE)
        control = 0
        itemPattern = re.compile("|".join((["^" + i for i in items])))
        for line in file:
            if control == 0:
                if itemPattern.search(line) is not None:
                    it = itemPattern.search(line).group(0)
                    control = 1
            else:
                if itemPattern.search(line) is not None:
                    dataList.append([it])
                    it = itemPattern.search(line).group(0)
                elif stop.search(line) is not None:
                    dataList.append([it])
                    break
                
        if dataList:
            data = pd.DataFrame.from_dict(dataList)
            data.columns = ['Element']
            data['Element'] = data['Element'].replace('\.','',regex=True)
            data = data[data['Element'] == "Item 103"]
            
            if data.empty:
                return(False)
            else:
                return(True)
            
        else: 
            pass
        
    # Alternative version to extract items text from 8-K files 
    # in which lines are not properly divided.
    def get_data_alternative(file):
        dataList = list()
        fullText = " ".join(file)
        fullText = unicodedata.normalize("NFKD", fullText).encode('ascii', 'ignore').decode('utf8')
        itemPattern = re.compile("\.\s*(Item\s[1-9][\.\d]*)")
        items = itemPattern.findall(fullText)
        itemsStart = list()
        stop = re.compile("SIGNATURE", re.IGNORECASE)
        sig = stop.search(fullText).start()
        for i in items:
            itStartPattern = re.compile("\.\s*"+i)
            itemsStart.append(itStartPattern.search(fullText).start())
        itemsStart.append(sig)
        n = 1
        while n < len(itemsStart) :
            dataList.append([items[(n-1)]])
            n += 1
            
        if dataList:
            data = pd.DataFrame.from_dict(dataList)
            data.columns = ['Element']
            data['Element'] = data['Element'].replace('\.','',regex=True)
            data = data[data['Element'] == "Item 103"]
            
            if data.empty:
                return(False)
            else:
                return(True)
        else:
            pass
          
    
    file = get_text(link)
    items = get_items(file)
    if len(items) >= 1:      
        return get_data(file, items)  
    else:
        try:
             return get_data_alternative(file)
        except:
            return False
    

In [ ]:
# dwownload the forms with the 1.03 Bankruptcy or Receivership section filled
dl = Downloader()

equity_ids = companyData['ticker']
for equity_id in equity_ids:
    dl.get("8-K", equity_id, amount=1, query="Bankruptcy or Receivership Chapter 11", after="2016-01-01", before="2022-12-31")

In [ ]:
# create a fuction to get the financial data from 10-K form with all necessary elements of financial reports 
def get_financial_data(cik, t):
    
    t_before = t - 3
    useful_facts = ['Assets', 'AccountsPayable', 'AssetsCurrent', 'AssetsNoncurrent', 'CashAndCashEquivalentsAtCarryingValue', 'EarningsPerShareBasic', 'GrossProfit', 'IncreaseDecreaseInAccountsPayable', 'InterestExpense', 'Liabilities', 'LiabilitiesAndStockholdersEquity', 'StockholdersEquity', 'LiabilitiesCurrent', 'LiabilitiesNoncurrent', 'LongTermDebt', 'NetCashProvidedByUsedInFinancingActivities', 'NetCashProvidedByUsedInInvestingActivities', 'NetCashProvidedByUsedInOperatingActivities', 'NetIncomeLoss', 'OperatingIncomeLoss', 'RetainedEarningsAccumulatedDeficit', 'Revenues']
    
    companyFacts = requests.get(
        f'https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json',
        headers=headers
    )
    
    financial_data = pd.DataFrame.from_dict((
                companyFacts.json()['facts']['us-gaap'][useful_facts[0]]['units']['USD']))
    financial_data = financial_data[financial_data['form'] == "10-K"]
        
    try:
        financial_data = financial_data.drop(columns=['start', 'accn', 'form', 'frame', 'fp', 'fy'])
    except KeyError:
        financial_data = financial_data.drop(columns=['accn', 'form', 'frame', 'fp', 'fy'])
        
    financial_data['end'] = pd.to_datetime(financial_data['end'], infer_datetime_format=True)
    financial_data['end'] = pd.to_numeric(financial_data['end'].dt.year)
    financial_data = financial_data[financial_data['end'] == t_before]
    financial_data = financial_data.sort_values('end').drop_duplicates('end',keep='last')
    financial_data.rename(columns={'val' : useful_facts[0]}, inplace=True)
    
    for fact in useful_facts[1:]:

        try: 
            fact_data = pd.DataFrame.from_dict((
               companyFacts.json()['facts']['us-gaap'][fact]['units']['USD']))
        except KeyError:
            try:
                fact_data = pd.DataFrame.from_dict((
                    companyFacts.json()['facts']['us-gaap'][fact]['units']['USD/shares']))
            except KeyError:
                continue
            
        fact_data = fact_data[fact_data['form'] == "10-K"]
        
        
        try:
            fact_data = fact_data.drop(columns=['start', 'accn', 'form', 'frame', 'fp', 'fy', 'filed'])
        except KeyError:
            fact_data = fact_data.drop(columns=['accn', 'form', 'frame', 'fp', 'fy', 'filed'])
        
        
        fact_data['end'] = pd.to_datetime(fact_data['end'], infer_datetime_format=True)
        fact_data['end'] = pd.to_numeric(fact_data['end'].dt.year)
        fact_data = fact_data[fact_data['end'] == t_before]
        fact_data = fact_data.sort_values('end').drop_duplicates('end',keep='last')
        fact_data.rename(columns={'val' : fact}, inplace=True)
        financial_data = financial_data.merge(fact_data, how='outer', on='end')
        
    financial_data['cik'] = cik
    
    return financial_data

In [ ]:
# now we need to look for 10-K filings for this companies and search for the texts of different section filings in this form
# Create new downloader object
d = Downloader()

# input the year range for filing data.
d.build_index_sec(2014, 2022)

# get the dataframe with all 8-K's filled in the range above
list_of_10k = d.find_files_by_type('10-K')

In [ ]:
# get the dataframe with all 10's filled in the range above
list_of_10k = d.find_files_by_type('10-K')

In [ ]:
# add leading zeros to CIK in order to perform future mappings
list_of_10k['CIK'] = list_of_10k['CIK'].astype(
                           str).str.zfill(10)

In [ ]:
list_of_10k

In [ ]:
# some data-transormation for the dataframe
list_of_10k.drop(columns=['Company Name', 'Form Type', 'url'], inplace=True)
list_of_10k = list_of_10k.merge(companyData, how='left', left_on='CIK', right_on='cik_str')
list_of_10k.drop(columns=['cik_str','title'], inplace=True)

In [ ]:
# we get the companies mapped as a companies with financial distress
fin_distress = pd.read_excel('financial_distress.xlsx')
fin_distress['year'] = pd.to_numeric(fin_distress['year'])

In [ ]:
# some additional transormation
list_of_10k['Date Filed'] = pd.to_datetime(list_of_10k['Date Filed'], infer_datetime_format=True)
list_of_10k['year'] = list_of_10k['Date Filed'].dt.year
list_of_10k['year'] = pd.to_numeric(list_of_10k['year'])

In [ ]:
list_of_10k

In [ ]:
# only companies with financial distress
fin_distress_10k = fin_distress.merge(list_of_10k, how='left', on=['ticker'])

In [ ]:
# we get only the rows with the year before filling the 8-K form to obtain MD&A text
fin_distress_10k = fin_distress_10k[fin_distress_10k['year_y'] == fin_distress_10k['year_x'] - 1]

In [ ]:
# get all the financial data for the companies in one dataframe
financial_data_fin_distress = get_financial_data(fin_distress_10k['CIK'].iloc[0], fin_distress_10k['year_x'].iloc[0])

for row in fin_distress_10k[1:].itertuples():
    try:
        buff = get_financial_data(row.CIK, row.year_x)
        financial_data_fin_distress = pd.concat([financial_data_fin_distress, buff])
    except:
        continue

In [ ]:
financial_data_fin_distress

In [ ]:
# merge witj initial dataframe to keep links to the forms
fin_distress_10k = financial_data_fin_distress.merge(fin_distress_10k, how='left', left_on='cik', right_on='CIK')

In [ ]:
# get the full link for the 10-K forms
sec_link = "https://www.sec.gov/Archives/"
fin_distress_10k['Filename'] = sec_link + fin_distress_10k['Filename'].astype(str)

In [ ]:
# create a function to get the text of MD&A section of 10K form for each observation
def parse_10k_filing(link, section):
    
    if section not in [0, 1, 2, 3]:
        print("Not a valid section")
        sys.exit()
    
    def get_text(link):
        page = requests.get(link, headers={'User-Agent': 'Mozilla'})
        html = bs(page.content, "lxml")
        text = html.get_text()
        text = unicodedata.normalize("NFKD", text).encode('ascii', 'ignore').decode('utf8')
        text = text.split("\n")
        text = " ".join(text)
        return(text)
    
    def extract_text(text, item_start, item_end):
        item_start = item_start
        item_end = item_end
        starts = [i.start() for i in item_start.finditer(text)]
        ends = [i.start() for i in item_end.finditer(text)]
        positions = list()
        for s in starts:
            control = 0
            for e in ends:
                if control == 0:
                    if s < e:
                        control = 1
                        positions.append([s,e])
        item_length = 0
        item_position = list()
        for p in positions:
            if (p[1]-p[0]) > item_length:
                item_length = p[1]-p[0]
                item_position = p

        item_text = text[item_position[0]:item_position[1]]

        return(item_text)

    text = get_text(link)
        
    if section == 1 or section == 0:
        try:
            item1_start = re.compile("item\s*[1][\.\;\:\-\_]*\s*\\b", re.IGNORECASE)
            item1_end = re.compile("item\s*1a[\.\;\:\-\_]\s*Risk|item\s*2[\.\,\;\:\-\_]\s*Prop", re.IGNORECASE)
            businessText = extract_text(text, item1_start, item1_end)
        except:
            businessText = "Something went wrong!"
        
    if section == 2 or section == 0:
        try:
            item1a_start = re.compile("(?<!,\s)item\s*1a[\.\;\:\-\_]\s*Risk", re.IGNORECASE)
            item1a_end = re.compile("item\s*2[\.\;\:\-\_]\s*Prop|item\s*[1][\.\;\:\-\_]*\s*\\b", re.IGNORECASE)
            riskText = extract_text(text, item1a_start, item1a_end)
        except:
            riskText = "Something went wrong!"
            
    if section == 3 or section == 0:
        try:
            item7_start = re.compile("item\s*[7][\.\;\:\-\_]*\s*\\bM", re.IGNORECASE)
            item7_end = re.compile("item\s*7a[\.\;\:\-\_]\sQuanti|item\s*8[\.\,\;\:\-\_]\s*", re.IGNORECASE)
            mdaText = extract_text(text, item7_start, item7_end)
        except:
            mdaText = "Something went wrong!"
    
    if section == 0:
        data = [businessText, riskText, mdaText]
    elif section == 1:
        data = [businessText]
    elif section == 2:
        data = [riskText]
    elif section == 3:
        data = [mdaText]
    return(data)

In [ ]:
# get the text using the function above
textual_data_fin_distress = pd.DataFrame(parse_10k_filing(fin_distress_10k['Filename'].iloc[0], 3), columns=['MD&A'])
textual_data_fin_distress['cik'] = fin_distress_10k['cik'].iloc[0]

for row in fin_distress_10k.itertuples():
    buff = pd.DataFrame(parse_10k_filing(row.Filename, 3), columns=['MD&A'])
    buff['cik'] = row.cik
    textual_data_fin_distress = pd.concat([textual_data_fin_distress, buff])

In [ ]:
# merge with initial dataframe
final_fin_distress = fin_distress_10k.merge(textual_data_fin_distress, how='left', on='cik')

In [ ]:
# parse news from the financial news website using special library. We use a VADER here for obtaininh the polarity scores and get the average score for the period before filling 8-K form
news_fin_distress = pd.DataFrame(data=(Sentiment(final_fin_distress['ticker'].iloc[0]).get_dataframe(days=4000)).mean()).T
news_fin_distress['ticker'] = final_fin_distress['ticker'].iloc[0]

# Get the headlines
# Returns a DataFrame with headlines, source and sentiment scores.

for row in final_fin_distress[1:].itertuples():
    try:
        buff = pd.DataFrame(data=(Sentiment(row.ticker).get_dataframe(days=4000)).mean()).T
        buff['ticker'] = row.ticker
        news_fin_distress = pd.concat([news_fin_distress, buff])
    except:
        continue

In [ ]:
news_fin_distress

In [ ]:
# obtain final df with polarity scores for the financial news
final_fin_distress = news_fin_distress.merge(final_fin_distress, how='left', on='ticker')

In [ ]:
# get the polarity score and subjectivity score for the MD&A
final_fin_distress['MD&A'] = final_fin_distress['MD&A'].astype(str)
final_fin_distress[['polarity', 'subjectivity']] = final_fin_distress['MD&A'].apply(lambda text: pd.Series(TextBlob(text).sentiment))

In [ ]:
# create some additional columns for the dataframe with financial indicators
final_fin_distress['D/E'] = final_fin_distress['Liabilities'] / final_fin_distress['StockholdersEquity']
final_fin_distress['D/A'] = final_fin_distress['Liabilities'] / final_fin_distress['Assets']
final_fin_distress['InterestCoverageRatio'] = final_fin_distress['OperatingIncomeLoss'] / final_fin_distress['InterestExpense']
final_fin_distress['D/EBIT'] = final_fin_distress['Liabilities'] / final_fin_distress['OperatingIncomeLoss']
final_fin_distress['CurrentRatio'] = final_fin_distress['AssetsCurrent'] / final_fin_distress['LiabilitiesCurrent']
final_fin_distress['CashRatio'] = final_fin_distress['CashAndCashEquivalentsAtCarryingValue'] / final_fin_distress['LiabilitiesCurrent']
final_fin_distress['ROE'] = final_fin_distress['NetIncomeLoss'] / final_fin_distress['StockholdersEquity']
final_fin_distress['ROA'] = final_fin_distress['NetIncomeLoss'] / final_fin_distress['Assets']
final_fin_distress['OperatingCashFlowRatio'] = final_fin_distress['NetCashProvidedByUsedInOperatingActivities'] / final_fin_distress['Liabilities']

In [ ]:
final_fin_distress

In [ ]:
#final_fin_distress.to_excel('final_fin_distress.xlsx')

In [ ]:
# now we need to get 10-K forms again for the companies that will not be classified as companies with financial distress
d = Downloader()
d.build_index_sec(2014, 2022)
list_of_10k = d.find_files_by_type('10-K')

In [ ]:
# get the dataframe with all 10's filled in the range above
list_of_10k = d.find_files_by_type('10-K')

In [ ]:
# add leading zeros to CIK in order to perform future mappings
list_of_10k['CIK'] = list_of_10k['CIK'].astype(
                           str).str.zfill(10)
list_of_10k.drop(columns=['Company Name', 'Form Type', 'url'], inplace=True)
list_of_10k = list_of_10k.merge(companyData, how='left', left_on='CIK', right_on='cik_str')
list_of_10k.drop(columns=['cik_str','title'], inplace=True)
list_of_10k['Date Filed'] = pd.to_datetime(list_of_10k['Date Filed'], infer_datetime_format=True)
list_of_10k['year'] = list_of_10k['Date Filed'].dt.year
list_of_10k['year'] = pd.to_numeric(list_of_10k['year'])

In [ ]:
# exclude the companies without the ticker
list_of_10k = list_of_10k[list_of_10k['ticker'].notnull()]

In [ ]:
# exclude the companies that had been already marked as financial distress
list_of_10k_out = list_of_10k[~list_of_10k['ticker'].isin(fin_distress['ticker'])]

In [ ]:
# get the latest information
list_of_10k_out.sort_values('Date Filed').groupby('ticker').tail(1)

In [ ]:
# get the subsample of the data
list_of_10k_out = list_of_10k_out.sample(n=500, random_state=1)

In [ ]:
sec_link = "https://www.sec.gov/Archives/"
list_of_10k_out['Filename'] = sec_link + list_of_10k_out['Filename'].astype(str)

In [ ]:
financial_data_normal = get_financial_data(list_of_10k_out['CIK'].iloc[0], list_of_10k_out['year'].iloc[0])

for row in list_of_10k_out[1:].itertuples():
    try:
        buff = get_financial_data(row.CIK, row.year)
        financial_data_normal = pd.concat([financial_data_normal, buff])
    except:
        continue

In [ ]:
list_of_10k_out = financial_data_normal.merge(list_of_10k_out, how='left', left_on='cik', right_on='CIK')

In [ ]:
list_of_10k_out

In [ ]:
# sorting by first name
list_of_10k_out.sort_values("cik", inplace=True)
  
# dropping ALL duplicate values
list_of_10k_out.drop_duplicates(subset="cik",
                     keep=False, inplace=True)
  
# displaying data
list_of_10k_out

In [ ]:
textual_data_normal = pd.DataFrame(parse_10k_filing(list_of_10k_out['Filename'].iloc[0], 3), columns=['MD&A'])
textual_data_normal['cik'] = list_of_10k_out['cik'].iloc[0]

for row in list_of_10k_out[1:].itertuples():
    buff = pd.DataFrame(parse_10k_filing(row.Filename, 3), columns=['MD&A'])
    buff['cik'] = row.cik
    textual_data_normal = pd.concat([textual_data_normal, buff])

In [ ]:
textual_data_normal

In [ ]:
final_normal = textual_data_normal.merge(list_of_10k_out, how='left', on='cik')

In [ ]:
news_normal = pd.DataFrame(data=(Sentiment(final_normal['ticker'].iloc[2]).get_dataframe(days=4000)).mean()).T
news_normal['ticker'] = final_normal['ticker'].iloc[2]

# Get the headlines
# Returns a DataFrame with headlines, source and sentiment scores.

for row in final_normal.itertuples():
    try:
        buff = pd.DataFrame(data=(Sentiment(row.ticker).get_dataframe(days=4000)).mean()).T
        buff['ticker'] = row.ticker
        news_normal = pd.concat([news_normal, buff])
    except:
        continue

In [ ]:
news_normal

In [ ]:
final_normal = news_normal.merge(final_normal, how='left', on='ticker')

In [ ]:
from textblob import TextBlob

final_normal['MD&A'] = final_normal['MD&A'].astype(str)
final_normal[['polarity', 'subjectivity']] = final_normal['MD&A'].apply(lambda text: pd.Series(TextBlob(text).sentiment))

In [ ]:
final_normal['D/E'] = final_normal['Liabilities'] / final_normal['StockholdersEquity']
final_normal['D/A'] = final_normal['Liabilities'] / final_normal['Assets']
final_normal['InterestCoverageRatio'] = final_normal['OperatingIncomeLoss'] / final_normal['InterestExpense']
final_normal['D/EBIT'] = final_normal['Liabilities'] / final_normal['OperatingIncomeLoss']
final_normal['CurrentRatio'] = final_normal['AssetsCurrent'] / final_normal['LiabilitiesCurrent']
final_normal['CashRatio'] = final_normal['CashAndCashEquivalentsAtCarryingValue'] / final_normal['LiabilitiesCurrent']
final_normal['ROE'] = final_normal['NetIncomeLoss'] / final_normal['StockholdersEquity']
final_normal['ROA'] = final_normal['NetIncomeLoss'] / final_normal['Assets']
final_normal['OperatingCashFlowRatio'] = final_normal['NetCashProvidedByUsedInOperatingActivities'] / final_normal['Liabilities']

In [ ]:
final_normal